In [4]:
from peft import LoraConfig, PeftModel
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, PeftModel 
import torch
from accelerate import FullyShardedDataParallelPlugin, Accelerator
#Load the base model with default precision
model_name = "teknium/OpenHermes-2.5-Mistral-7B"
adapter = "./mistral/checkpoint-20_800_1024"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                            # max_memory={1: "40GiB", 2: "40GiB", "cpu": "30GiB"},
                                            )

#Load and activate the adapter on top of the base model
model = PeftModel.from_pretrained(model, adapter)

#Merge the adapter with the base model
model = model.merge_and_unload()

#Save the merged model in a directory in the safetensors format
model_dir = "./mistral/merged_model_20/"
model.save_pretrained(model_dir, safe_serialization=True)

#Save the custom tokenizer in the same directory
tokenizer.save_pretrained(model_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|████████████████████████████████████████| 2/2 [00:07<00:00,  3.65s/it]


('./mistral/merged_model_20/tokenizer_config.json',
 './mistral/merged_model_20/special_tokens_map.json',
 './mistral/merged_model_20/tokenizer.model',
 './mistral/merged_model_20/added_tokens.json',
 './mistral/merged_model_20/tokenizer.json')